In [1]:
!pip install torch
!pip install transformers
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd

# f = pd.read_json(file_path, lines=True)
df_image_train = pd.read_json("data/train.jsonl", lines=True)
df_image_val = pd.read_json("data/dev.jsonl", lines=True)
df_image_test = pd.read_json("data/test.jsonl", lines=True)
df_image_train.head()

,id,img,label,text
0,42953,img/42953.png,0,its their character not their color that matters
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...
2,13894,img/13894.png,0,putting bows on your pet
3,37408,img/37408.png,0,i love everything and everybody! except for sq...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h..."


In [3]:
pip install --upgrade transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
print(df_image_train.head())

      id            img  label  \
0  42953  img/42953.png      0   
1  23058  img/23058.png      0   
2  13894  img/13894.png      0   
3  37408  img/37408.png      0   
4  82403  img/82403.png      0   

                                                text  
0   its their character not their color that matters  
1  don't be afraid to love again everyone is not ...  
2                           putting bows on your pet  
3  i love everything and everybody! except for sq...  
4  everybody loves chocolate chip cookies, even h...  


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torchvision.models as models
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from PIL import Image
import pandas as pd
import os
import torchvision.transforms as T

# Transformers model for text
text_model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Torchvision model for images
image_model = models.resnet101(pretrained=True)
image_model = nn.Sequential(*list(image_model.children())[:-1])  # Remove the last FC layer


def collate_fn(batch):
    images, texts, labels = zip(*batch)

    # Pad sequences
    input_ids = pad_sequence([t['input_ids'] for t in texts])
    attention_mask = pad_sequence([t['attention_mask'] for t in texts])

    # Convert to tensors
    input_ids = torch.as_tensor(input_ids, dtype=torch.long)
    attention_mask = torch.as_tensor(attention_mask, dtype=torch.long)
    labels = torch.as_tensor(labels, dtype=torch.long)

    # Transpose image tensor (batch_size should be first)
    images = torch.stack(images).transpose(0, 1)
    
    return images, {'input_ids': input_ids, 'attention_mask': attention_mask}, labels


class HatefulMemesDataset(Dataset):
    def __init__(self, json_file, img_dir, transforms=None):
        self.data = pd.read_json(json_file, lines=True)
        self.img_dir = img_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_id = self.data.loc[idx, 'img']
        img_path = os.path.join(self.img_dir, img_id)
        image = Image.open(img_path).convert('RGB')

        if self.transforms:
            image = self.transforms(image)
        
        # Process text data
        text = self.data.loc[idx, 'text']
        text = tokenizer(text, return_tensors='pt', padding="max_length", truncation=True, max_length=64)

        label = self.data.loc[idx, 'label']

        return image, text, label


# Define your transforms
transforms = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # normalization settings for resnet models
])

# Initialize your dataset
dataset = HatefulMemesDataset(json_file="data/train.jsonl",
                              img_dir="data",
                              transforms=transforms)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# # Define a simple model for classification
class CombinedModel(nn.Module):
    def __init__(self, text_model, image_model):
        super(CombinedModel, self).__init__()
        self.text_model = text_model
        self.image_model = image_model
        self.classifier = nn.Linear(text_model.config.hidden_size + 2048, 2)  # 2048 is the output size of resnet101

    def forward(self, image, input_ids, attention_mask):
        text_features = self.text_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        image_features = self.image_model(image).view(image.size(0), -1)
        combined = torch.cat((text_features, image_features), dim=1)
        logits = self.classifier(combined)
        return logits


# Create your model
model = CombinedModel(text_model, image_model)

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 10
model.train()
for epoch in range(num_epochs):
    for images, texts, labels in dataloader:
        # Move inputs and targets to the same device as the model
        images = images.permute(1, 0, 2, 3)
        images = images.to(device)
        input_ids = texts['input_ids'].squeeze().to(device)
#         print("text attention mask", texts['attention_mask'])
        attention_mask = texts['attention_mask'].squeeze().to(device)
        labels = labels.to(device)

        # Forward pass
        logits = model(images, input_ids, attention_mask)

        # Compute the loss
        loss = criterion(logits, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch: {epoch}, Loss: {loss.item()}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/opt/conda/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and

Epoch: 0, Loss: 0.5849014520645142
Epoch: 1, Loss: 0.6733682155609131
Epoch: 2, Loss: 0.9221835732460022
Epoch: 3, Loss: 0.4037746787071228
Epoch: 4, Loss: 0.5543085932731628
Epoch: 5, Loss: 0.8913806080818176
Epoch: 6, Loss: 0.7937508821487427


In [ ]:
# from transformers import AutoTokenizer, AutoModel
# import torchvision.models as models
# import torch
# from torch import nn, optim
# from torch.utils.data import Dataset, DataLoader
# from torch.nn.utils.rnn import pad_sequence
# from PIL import Image
# import pandas as pd
# import os
# import torchvision.transforms as T

# # Transformers model for text
# text_model = AutoModel.from_pretrained("bert-base-uncased")
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# # Torchvision model for images
# image_model = models.resnet101(pretrained=True)
# image_model = nn.Sequential(*list(image_model.children())[:-1])  # Remove the last FC layer


# def collate_fn(batch):
#     images, texts, labels = zip(*batch)

#     # Pad sequences
#     input_ids = pad_sequence([t['input_ids'] for t in texts])
#     attention_mask = pad_sequence([t['attention_mask'] for t in texts])

#     # Convert to tensors
#     input_ids = torch.as_tensor(input_ids, dtype=torch.long)
#     attention_mask = torch.as_tensor(attention_mask, dtype=torch.long)
#     labels = torch.as_tensor(labels, dtype=torch.long)

#     # Transpose image tensor (batch_size should be first)
#     images = torch.stack(images).transpose(0, 1)
    
#     return images, {'input_ids': input_ids, 'attention_mask': attention_mask}, labels


# class HatefulMemesDataset(Dataset):
#     def __init__(self, json_file, img_dir, transforms=None):
#         self.data = pd.read_json(json_file, lines=True)
#         self.img_dir = img_dir
#         self.transforms = transforms

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         img_id = self.data.loc[idx, 'img']
#         img_path = os.path.join(self.img_dir, img_id)
#         image = Image.open(img_path).convert('RGB')

#         if self.transforms:
#             image = self.transforms(image)
        
#         # Process text data
#         text = self.data.loc[idx, 'text']
#         text = tokenizer(text, return_tensors='pt', padding="max_length", truncation=True, max_length=64)

#         label = self.data.loc[idx, 'label']

#         return image, text, label


# # Define your transforms
# transforms = T.Compose([
#     T.Resize((224, 224)),
#     T.ToTensor(),
#     T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # normalization settings for resnet models
# ])

# # Initialize your dataset
# dataset = HatefulMemesDataset(json_file="data/train.jsonl",
#                               img_dir="data",
#                               transforms=transforms)

# # Create DataLoader
# dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# # # Define a simple model for classification
# class CombinedModel(nn.Module):
#     def __init__(self, text_model, image_model):
#         super(CombinedModel, self).__init__()
#         self.text_model = text_model
#         self.image_model = image_model
#         self.classifier = nn.Linear(text_model.config.hidden_size + 2048, 2)  # 2048 is the output size of resnet101

#     def forward(self, image, input_ids, attention_mask):
#         text_features = self.text_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
#         image_features = self.image_model(image).view(image.size(0), -1)
#         combined = torch.cat((text_features, image_features), dim=1)
#         logits = self.classifier(combined)
#         return logits


# # Create your model
# # model = CombinedModel(text_model, image_model)

# # Move to GPU if available
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# image_model = image_model.to(device)

# text_model = text_model.to(device)

# # Loss and Optimizer
# criterion = nn.BinaryCrossEntropyLoss()
# text_optimizer = optim.Adam(model.parameters(), lr=0.0001)
# image_optimizer = optim.Adam(model.parameters(), lr=0.0001)

# # Training loop
# num_epochs = 10
# model.train()
# for epoch in range(num_epochs):
#     for images, texts, labels in dataloader:
#         # Move inputs and targets to the same device as the model
#         images = images.permute(1, 0, 2, 3)
#         images = images.to(device)
#         input_ids = texts['input_ids'].squeeze().to(device)
# #         print("text attention mask", texts['attention_mask'])
#         attention_mask = texts['attention_mask'].squeeze().to(device)
#         labels = labels.to(device)

#         # Forward pass
# #         logits = model(images, input_ids, attention_mask)
#         text_features = text_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
#         image_features = image_model(image).view(image.size(0), -1)
#         combined = torch.cat((text_features, image_features), dim=1)

#         # Compute the loss
#         loss = criterion(logits, labels)

#         # Backward pass and optimization
#         loss.backward()
#         text_optimizer.step()
        
#         optimizer.zero_grad()

#     print(f"Epoch: {epoch}, Loss: {loss.item()}")